In [4]:
# =========================================================
# 0) Imports & Config
# =========================================================
import os, sys, time, random, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import scanpy as sc
from scipy.sparse import issparse, csr_matrix

import sys, os
sys.path.append("/home1/jijh/diffusion_project/git_repo/GoldSpace")

from src.spaglam_preproc.utils.hest_loading import HESTDataset


# ---------- 全局配置 ----------
DATA_DIR = "/cwStorage/nodecw_group/jijh/hest_1k"
OUT_BASE = "/cwStorage/nodecw_group/jijh/hest_sentences_human_all_second"
CACHE_DIR = os.path.join(OUT_BASE, "cache")
os.makedirs(CACHE_DIR, exist_ok=True)

CACHE_H5AD = os.path.join(CACHE_DIR, "adata_merged_preprocessed_canonical_collapsed.h5ad")
HVG_TXT = os.path.join(OUT_BASE, "global_hvgs.txt")

SAMPLES_TO_EXCLUDE = {
    'TENX15','MEND16','MEND15','MEND14','MEND13','MEND12',
    'MEND11','MEND10','MEND9','MEND8','MEND7','MEND2',
    'MEND1','NCBI657','NCBI814'
}

SEED = 42
random.seed(SEED); np.random.seed(SEED)

N_TOP_HVGS = 5000
N_TOP_GENES_IN_SENT = 50
BATCH_KEY = 'sample_id'

print(f"🌱 Seed={SEED}; 输出目录={OUT_BASE}")




🌱 Seed=42; 输出目录=/cwStorage/nodecw_group/jijh/hest_sentences_human_all_second


In [5]:

# =========================================================
# 1) Utilities
# =========================================================
def format_time(seconds: float) -> str:
    m, s = divmod(seconds, 60)
    return f"{int(m)}分 {s:.2f}秒"

def load_sample_data(sample, batch_key: str = 'sample_id'):
    """把单个 HESTSample 加载到内存, 标注 obs_names 与 batch_key."""
    try:
        sample.load_st_data(lazy=False)
        if sample.adata is None:
            return None
        ad = sample.adata.to_memory()
        # obs_names 加前缀避免不同样本重复
        ad.obs_names = [f"{sample.sample_id}_{x}" for x in ad.obs_names]
        ad.obs[batch_key] = sample.sample_id
        return ad
    except Exception as e:
        print(f"Error loading {sample.sample_id}: {e}")
        return None

def try_load_canonical_gene_set(cache_dir: str):
    """尝试从 cache_dir 导入 human_gene_symbols.py；若失败返回 None."""
    try:
        sys.path.insert(0, cache_dir)
        from human_gene_symbols import human_gene_symbols
        return set([g.upper() for g in human_gene_symbols])
    except Exception as e:
        print(f"⚠️ 未找到或导入失败: human_gene_symbols.py；将仅做聚合不做权威过滤。原因: {e}")
        return None
    finally:
        if sys.path[0] == cache_dir:
            sys.path.pop(0)

def canonicalize_var_names_to_upper(adata: sc.AnnData):
    """去前缀/版本号，并统一大写，便于与 HGNC 对齐。"""
    cleaned = []
    for name in adata.var_names:
        base = name.replace('GRCh38______','').split('.')[0]
        cleaned.append(base.upper())
    adata.var_names = cleaned
    return adata

def collapse_duplicate_genes(adata: sc.AnnData) -> sc.AnnData:
    """
    聚合同名基因（按列求和），避免 var_names_make_unique() 带来的 -1/-2 后缀。
    稀疏/稠密均支持；保留 obs/obsm/uns。
    """
    names = np.asarray(adata.var_names)
    uniq, inv = np.unique(names, return_inverse=True)
    if uniq.size == names.size:
        return adata  # 无重复，无需聚合

    # 构造列映射稀疏矩阵 G (n_vars x n_unique)
    ones = np.ones(inv.size, dtype=np.float32)
    G = csr_matrix((ones, (np.arange(inv.size), inv)), shape=(inv.size, uniq.size))

    if issparse(adata.X):
        X_new = adata.X @ G
    else:
        # dense @ sparse -> dense ndarray
        X_new = np.asarray(adata.X) @ G

    ad_new = sc.AnnData(X_new, obs=adata.obs.copy(), var=pd.DataFrame(index=uniq))
    ad_new.obsm = adata.obsm.copy()
    ad_new.uns = adata.uns.copy()
    return ad_new

def enforce_canonical_and_collapse(adata: sc.AnnData, canonical_genes_upper: set | None):
    """
    1) 规范基因名（去前缀/版本号并大写）
    2) （可选）基于权威表过滤
    3) 聚合同名基因（列求和）
    """
    adata = canonicalize_var_names_to_upper(adata)

    if canonical_genes_upper is not None:
        keep = pd.Index(adata.var_names).isin(list(canonical_genes_upper))
        before = adata.n_vars
        adata = adata[:, keep].copy()
        print(f"✅ 权威过滤: 基因 {before} → {adata.n_vars}")

    # 聚合同名（核心修复）
    before = adata.n_vars
    adata = collapse_duplicate_genes(adata)
    print(f"✅ 聚合同名: 基因 {before} → {adata.n_vars}（去掉 -1/-2 后缀的根源）")
    # 最终保证无重复且不做“唯一化”加后缀
    assert not pd.Index(adata.var_names).duplicated().any(), "聚合后仍有重复基因名"

    return adata

def qc_and_basic_filters(adata: sc.AnnData) -> sc.AnnData:
    """细胞/spot 过滤 + 基因过滤 + 线粒体占比过滤."""
    ad = adata.copy()
    ad.var['mt'] = ad.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(ad, qc_vars=['mt'], inplace=True, percent_top=None, log1p=False)

    n0 = ad.n_obs; p0 = ad.n_vars
    sc.pp.filter_cells(ad, min_genes=200)
    ad = ad[ad.obs.pct_counts_mt < 20, :].copy()

    min_cells = max(1, int(ad.n_obs * 0.001))  # 至少 0.1% 的 spots；且 ≥1
    sc.pp.filter_genes(ad, min_cells=min_cells)

    print(f"✅ QC后: spots {n0}→{ad.n_obs}, genes {p0}→{ad.n_vars} (min_cells={min_cells})")
    if 'in_tissue' in ad.obs:
        ad.obs['in_tissue'] = ad.obs['in_tissue'].fillna(0).astype(bool)
    return ad

def normalize_log1p(adata: sc.AnnData) -> sc.AnnData:
    ad = adata.copy()
    sc.pp.normalize_total(ad, target_sum=1e4)
    sc.pp.log1p(ad)
    return ad

def compute_hvgs(adata: sc.AnnData, n_top: int = 5000, batch_key: str = 'sample_id',
                 flavor: str = 'seurat_v3_paper') -> pd.Index:
    sc.pp.highly_variable_genes(adata, n_top_genes=n_top, batch_key=batch_key, flavor=flavor)
    hvgs = adata.var.index[adata.var['highly_variable']]
    return hvgs

def validate_hvgs(hvgs: pd.Index, canonical_genes_upper: set | None, n_expected: int):
    # 1) 无重复
    assert not pd.Index(hvgs).duplicated().any(), "HVG 列表存在重复基因名"
    # 2) 无 -数字 后缀
    bad = [g for g in hvgs if re.search(r"-\d+$", g)]
    assert len(bad) == 0, f"HVG 中仍含非标准名（如 -1 后缀）: {bad[:5]} ..."
    # 3) （可选）全部在权威集合
    if canonical_genes_upper is not None:
        outside = [g for g in hvgs if g.upper() not in canonical_genes_upper]
        assert len(outside) == 0, f"HVG 中存在不在权威集合的基因（前若干）：{outside[:5]} ..."
    # 4) 数量
    if len(hvgs) != n_expected:
        print(f"⚠️ HVG 数量 {len(hvgs)} != 期望 {n_expected}。若强过滤后基因不足, 这是可解释的。")

def save_hvgs(hvgs: pd.Index, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w') as f:
        for g in hvgs:
            f.write(f"{g}\n")
    print(f"💾 HVG 列表已保存: {path}")

# --------- 句子生成相关 ----------
def topk_indices(x: np.ndarray, k: int):
    """用 argpartition 快速取 top-k 索引（降序）。"""
    k = min(k, x.size)
    if k <= 0:
        return np.array([], dtype=int)
    idx = np.argpartition(x, -k)[-k:]
    return idx[np.argsort(x[idx])[::-1]]

def make_sentences_for_sample(sample_id: str, adata_sample: sc.AnnData,
                              hvgs: pd.Index, out_base: str,
                              k: int = 50):
    out_dir = os.path.join(out_base, f"{sample_id}_sentences_hvg")
    os.makedirs(out_dir, exist_ok=True)

    ad = adata_sample[:, adata_sample.var_names.isin(hvgs)].copy()
    if ad.n_vars == 0:
        print(f"⚠️ {sample_id}: 无 HVG 交集, 跳过")
        return

    obs = ad.obs.copy()
    genes = ad.var_names.to_numpy()

    # 坐标鲁棒获取
    if 'pxl_col_in_fullres' not in obs.columns or 'pxl_row_in_fullres' not in obs.columns:
        if 'spatial' in ad.obsm:
            obs['pxl_col_in_fullres'] = ad.obsm['spatial'][:, 0]
            obs['pxl_row_in_fullres'] = ad.obsm['spatial'][:, 1]
        else:
            obs['pxl_col_in_fullres'] = np.arange(ad.n_obs)
            obs['pxl_row_in_fullres'] = 0

    for i in range(ad.n_obs):
        r, c = obs.iloc[i]['pxl_row_in_fullres'], obs.iloc[i]['pxl_col_in_fullres']
        if pd.isna(r) or pd.isna(c):
            suffix = f"{i}_0_no_coord"
        else:
            suffix = f"{int(round(r))}_{int(round(c))}"
        path = os.path.join(out_dir, f"{sample_id}_{suffix}.txt")
        if os.path.exists(path):
            continue

        # 取表达向量
        if issparse(ad.X):
            vec = ad.X.getrow(i).toarray().ravel()
        else:
            vec = np.asarray(ad.X[i]).ravel()

        idx = topk_indices(vec, k)
        toks = genes[idx]
        with open(path, "w") as f:
            f.write(" ".join(toks))

def validate_sentences_random(hvgs: set, out_base: str, sample_ids: list, n_checks: int = 20):
    """随机抽样句子文件，验证所有 token 均∈ HVG."""
    import random, os
    sample_ids = sample_ids[:] if len(sample_ids) <= n_checks else random.sample(sample_ids, n_checks)
    ok = 0; bad = []
    for sid in sample_ids:
        d = os.path.join(out_base, f"{sid}_sentences_hvg")
        if not (os.path.exists(d) and os.listdir(d)):
            continue
        fname = random.choice(os.listdir(d))
        with open(os.path.join(d, fname), 'r') as f:
            toks = f.read().split()
        if all(t in hvgs for t in toks):
            ok += 1
        else:
            bad.append((sid, fname, [t for t in toks if t not in hvgs]))
    print(f"🔍 句子验证：通过 {ok}/{len(sample_ids)} 个样本。")
    if bad:
        print("❌ 非HVG token 示例（最多显示5例）：")
        for sid, fn, notin in bad[:5]:
            print(f"  - {sid}/{fn}: {notin[:10]} ...")

# =========================================================
# 2) Main: 加载→合并→清洗→聚合→QC→HVG→句子
# =========================================================
t0 = time.time()

# 2.1 读取样本
print("初始化 HESTDataset...")
dataset = HESTDataset(data_dir=DATA_DIR)
samples = [s for s in dataset.get_samples(species="Homo sapiens")
           if s.sample_id not in SAMPLES_TO_EXCLUDE]
print(f"有效样本数: {len(samples)}（已排除: {len(SAMPLES_TO_EXCLUDE)}）")

# 2.2 合并（带 batch_key）
adatas = []
for s in tqdm(samples, desc="加载样本"):
    ad = load_sample_data(s, batch_key=BATCH_KEY)
    if ad is None: 
        continue
    # 样本内如有重复列，先做一次简单去重（保守做法，最终还会统一聚合）
    if pd.Index(ad.var_names).duplicated().any():
        ad = ad[:, ~pd.Index(ad.var_names).duplicated()].copy()
    adatas.append(ad)
if not adatas:
    raise ValueError("未能加载任何 AnnData")

adata = sc.concat(adatas, join='outer', uns_merge='unique')
print(f"合并完成: {adata.shape}")


初始化 HESTDataset...
有效样本数: 634（已排除: 15）


加载样本:   0%|          | 0/634 [00:00<?, ?it/s]

/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/gigapath/lib/python3.1

合并完成: (1351225, 220010)


# 利用hugo

In [16]:
# 0) Imports & Config
# =========================================================
import os, sys, time, random, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import scanpy as sc
from scipy.sparse import issparse, csr_matrix


# ---------- 全局配置 ----------
DATA_DIR = "/cwStorage/nodecw_group/jijh/hest_1k"
OUT_BASE = "/cwStorage/nodecw_group/jijh/hest_sentences_hugo"
CACHE_DIR = os.path.join(OUT_BASE, "cache")
os.makedirs(CACHE_DIR, exist_ok=True)

CACHE_H5AD = os.path.join(CACHE_DIR, "adata_merged_preprocessed_canonical_collapsed.h5ad")
HVG_TXT = os.path.join(OUT_BASE, "global_hvgs.txt")

# ★★ NEW: HGNC 官方全表路径（你刚给的 realpath）
HGNC_TSV = "/public/home/jijh/diffusion_project/git_repo/GoldSpace/hgnc_complete_set.txt"

# （可选）过滤条件
KEEP_STATUS = {"Approved"}  # 只保留“已批准”命名
# KEEP_LOCUS_TYPES = {"protein-coding gene"}  # 若只保留蛋白编码基因，解开此行
KEEP_LOCUS_TYPES = None  # 不限定类型时用 None

SAMPLES_TO_EXCLUDE = {
    'TENX15','MEND16','MEND15','MEND14','MEND13','MEND12',
    'MEND11','MEND10','MEND9','MEND8','MEND7','MEND2',
    'MEND1','NCBI657','NCBI814'
}

SEED = 42
random.seed(SEED); np.random.seed(SEED)

N_TOP_HVGS = 5000
N_TOP_GENES_IN_SENT = 50
BATCH_KEY = 'sample_id'

print(f"Seed={SEED}; 输出目录={OUT_BASE}")

# =========================================================
# 1) Utilities
# =========================================================
def format_time(seconds: float) -> str:
    m, s = divmod(seconds, 60)
    return f"{int(m)}分 {s:.2f}秒"

def load_sample_data(sample, batch_key: str = 'sample_id'):
    """把单个 HESTSample 加载到内存, 标注 obs_names 与 batch_key."""
    try:
        sample.load_st_data(lazy=False)
        if sample.adata is None:
            return None
        ad = sample.adata.to_memory()
        ad.obs_names = [f"{sample.sample_id}_{x}" for x in ad.obs_names]  # 避免 obs_names 冲突
        ad.obs[batch_key] = sample.sample_id
        return ad
    except Exception as e:
        print(f"Error loading {sample.sample_id}: {e}")
        return None

# ---------- NEW: 载入 HGNC 官方命名资源 ----------
def load_hgnc_resources(tsv_path: str,
                        keep_status: set[str] | None = None,
                        keep_locus_types: set[str] | None = None):
    """
    从 HGNC 完整表构建：
    - canonical_set：当前官方符号集合（大写）
    - synonym_map：别名/旧名（大写）到官方符号（大写）的映射
    - symbol2type：官方符号 -> locus_type
    """
    # HGNC 文件是 TSV（制表符分隔）
    df = pd.read_csv(tsv_path, sep='\t', dtype=str, na_filter=False)
    # 标准列名（不同版本表头一致：symbol, alias_symbol, prev_symbol, status, locus_type）
    # 统一大写符号
    df['symbol_u'] = df['symbol'].str.upper()

    # 过滤 status
    if keep_status:
        df = df[df['status'].isin(keep_status)].copy()

    # 过滤 locus_type（可选）
    if keep_locus_types:
        df = df[df['locus_type'].isin(keep_locus_types)].copy()

    # 官方集合与类型映射
    canonical_set = set(df['symbol_u'])
    symbol2type = dict(zip(df['symbol_u'], df['locus_type']))

    # 构建同义名/旧名映射
    def _split_multi(s: str):
        if not s:
            return []
        s = s.strip().strip('"')
        parts = re.split(r'[|,;/]\s*', s)
        return [p for p in parts if p]

    synonym_map = {}
    for _, r in df.iterrows():
        sym = r['symbol_u']
        # 仅用 alias_symbol / prev_symbol，避免用 name/alias_name 造成误映射
        for col in ('alias_symbol', 'prev_symbol'):
            if col in r and r[col]:
                for a in _split_multi(r[col]):
                    synonym_map[a.upper()] = sym

    print(f"HGNC 载入：官方符号 {len(canonical_set)} 个；同义/旧名映射 {len(synonym_map)} 条。")
    return canonical_set, synonym_map, symbol2type

# ---------- 名称规范化 + 同义名映射 + 列聚合 ----------
def canonicalize_var_names_to_upper(adata: sc.AnnData):
    """去前缀/版本号，并统一大写，便于与 HGNC 对齐。"""
    cleaned = []
    for name in adata.var_names:
        base = name.replace('GRCh38______','').split('.')[0]
        cleaned.append(base.upper())
    adata.var_names = cleaned
    return adata

def map_synonyms_to_official(adata: sc.AnnData, synonym_map: dict[str, str]) -> tuple[sc.AnnData, int]:
    """把 var_names 中的同义/旧名映射为官方符号；返回映射数量。"""
    if not synonym_map:
        return adata, 0
    names = list(adata.var_names)
    n_map = 0
    for i, g in enumerate(names):
        if g in synonym_map:
            names[i] = synonym_map[g]
            n_map += 1
    adata.var_names = names
    return adata, n_map

def collapse_duplicate_genes(adata: sc.AnnData) -> sc.AnnData:
    """
    聚合同名基因（按列求和），避免 var_names_make_unique() 带来的 -1/-2 后缀。
    稀疏/稠密均支持；保留 obs/obsm/uns。
    """
    names = np.asarray(adata.var_names)
    uniq, inv = np.unique(names, return_inverse=True)
    if uniq.size == names.size:
        return adata  # 无重复

    ones = np.ones(inv.size, dtype=np.float32)
    G = csr_matrix((ones, (np.arange(inv.size), inv)), shape=(inv.size, uniq.size))

    if issparse(adata.X):
        X_new = adata.X @ G
    else:
        X_new = np.asarray(adata.X) @ G

    ad_new = sc.AnnData(X_new, obs=adata.obs.copy(), var=pd.DataFrame(index=uniq))
    ad_new.obsm = adata.obsm.copy()
    ad_new.uns = adata.uns.copy()
    return ad_new

def enforce_hgnc_and_collapse(adata: sc.AnnData,
                              canonical_set: set[str] | None,
                              synonym_map: dict[str, str] | None,
                              audit_dir: str):
    """
    1) 规范化（去前缀/版本号 + 大写）
    2) 同义名/旧名 → 官方符号
    3) （可选）只保留官方集合
    4) 聚合同名（列求和）
    并输出审计报告：映射了多少、被过滤多少、未命中多少。
    """
    os.makedirs(audit_dir, exist_ok=True)

    # 0) 记录原始名
    before_names = pd.Index(adata.var_names)

    # 1) 统一 + 去版本号
    ad = canonicalize_var_names_to_upper(adata.copy())

    # 2) 同义名映射
    mapped_n = 0
    if synonym_map:
        ad, mapped_n = map_synonyms_to_official(ad, synonym_map)

    # 3) 官方集合过滤（可选）
    if canonical_set:
        keep_mask = pd.Index(ad.var_names).isin(list(canonical_set))
        kept = int(keep_mask.sum()); dropped = int((~keep_mask).sum())
        ad = ad[:, keep_mask].copy()
    else:
        kept = ad.n_vars; dropped = 0

    # 4) 聚合同名（核心修复，杜绝 -1/-2）
    ad = collapse_duplicate_genes(ad)

    # ---- 审计输出 ----
    after_names = pd.Index(ad.var_names)
    df_audit = pd.DataFrame({
        "stage": ["init", "after_canon+map+filter+collapse"],
        "n_genes": [len(before_names), len(after_names)],
        "n_mapped_from_alias_prev": [mapped_n, mapped_n],
        "n_filtered_by_canonical": [dropped, dropped]
    })
    df_audit.to_csv(os.path.join(audit_dir, "gene_name_audit.tsv"), sep='\t', index=False)

    # 列出未命中官方集合/未被映射者（仅采样展示）
    if canonical_set:
        unresolved = sorted(set(g for g in before_names.str.upper()
                                if (g not in canonical_set) and (g not in synonym_map)))
        sample_unresolved = unresolved[:20]
        with open(os.path.join(audit_dir, "unresolved_non_hgnc_samples.txt"), "w") as f:
            for g in sample_unresolved:
                f.write(g + "\n")

    print(f"✅ 名称对齐完成：映射 {mapped_n}；过滤 {dropped}；最终基因 {ad.n_vars}")
    assert not pd.Index(ad.var_names).duplicated().any(), "聚合后仍有重复基因名"
    return ad

def qc_and_basic_filters(adata: sc.AnnData) -> sc.AnnData:
    """细胞/spot 过滤 + 基因过滤 + 线粒体占比过滤."""
    ad = adata.copy()
    ad.var['mt'] = ad.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(ad, qc_vars=['mt'], inplace=True, percent_top=None, log1p=False)

    n0 = ad.n_obs; p0 = ad.n_vars
    sc.pp.filter_cells(ad, min_genes=200)
    ad = ad[ad.obs.pct_counts_mt < 20, :].copy()

    min_cells = max(1, int(ad.n_obs * 0.001))  # ≥1
    sc.pp.filter_genes(ad, min_cells=min_cells)

    print(f"✅ QC后: spots {n0}→{ad.n_obs}, genes {p0}→{ad.n_vars} (min_cells={min_cells})")
    if 'in_tissue' in ad.obs:
        ad.obs['in_tissue'] = ad.obs['in_tissue'].fillna(0).astype(bool)
    return ad

def normalize_log1p(adata: sc.AnnData) -> sc.AnnData:
    ad = adata.copy()
    sc.pp.normalize_total(ad, target_sum=1e4)
    sc.pp.log1p(ad)
    return ad

def compute_hvgs(adata: sc.AnnData, n_top: int = 5000, batch_key: str = 'sample_id',
                 flavor: str = 'seurat_v3_paper') -> pd.Index:
    sc.pp.highly_variable_genes(adata, n_top_genes=n_top, batch_key=batch_key, flavor=flavor)
    hvgs = adata.var.index[adata.var['highly_variable']]
    return hvgs

# ---------- HVG 验证：对照 HGNC + 类型概览 ----------
def validate_hvgs(hvgs: pd.Index,
                  canonical_set: set[str] | None,
                  symbol2type: dict[str, str] | None,
                  n_expected: int):
    # 1) 无重复
    assert not pd.Index(hvgs).duplicated().any(), "HVG 列表存在重复基因名"

    # 2) 与官方集合对齐（主判据）
    outside = []
    if canonical_set:
        # 注意：我们的管线已统一大写；为稳妥仍使用 upper() 对齐
        outside = [g for g in hvgs if g.upper() not in canonical_set]
        if outside:
            print(f"⚠️ 有 {len(outside)} 个 HVG 不在 HGNC 官方集合（示例）：{outside[:10]}")

    # 3) 仅对“不在官方集合”的基因再做后缀可疑性检测
    suspicious_suffix = [g for g in outside if re.search(r"-\d+$", g)]
    # —— 如果你希望对这类“强疑似 make_unique 伪后缀”严苛处理，用 assert；否则改成 warning：
    assert len(suspicious_suffix) == 0, (
        "HVG 中存在疑似唯一化伪后缀（不在HGNC且形如 `-数字`）的基因："
        f"{suspicious_suffix[:10]}"
    )

    # 4) 可选：locus_type 分布，便于审计（信息性输出，不做强校验）
    if symbol2type:
        types = pd.Series([symbol2type.get(g.upper(), "UNK") for g in hvgs]).value_counts()
        types.to_csv(os.path.join(CACHE_DIR, "hvg_locus_type_distribution.tsv"),
                     sep='\t', header=['count'])
        print("HVG locus_type 分布已写入 cache/hvg_locus_type_distribution.tsv")

    # 5) 数量提醒（不强制）
    if len(hvgs) != n_expected:
        print(f"⚠️ HVG 数量 {len(hvgs)} != 期望 {n_expected}。若前面做了严格过滤，这可能是正常的。")


def save_hvgs(hvgs: pd.Index, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w') as f:
        for g in hvgs:
            f.write(f"{g}\n")
    print(f"HVG 列表已保存: {path}")

# --------- 句子生成相关 ----------
def topk_indices(x: np.ndarray, k: int):
    """argpartition 取 top-k（降序），比全排序更快。"""
    k = min(k, x.size)
    if k <= 0:
        return np.array([], dtype=int)
    idx = np.argpartition(x, -k)[-k:]
    return idx[np.argsort(x[idx])[::-1]]

def make_sentences_for_sample(sample_id: str, adata_sample: sc.AnnData,
                              hvgs: pd.Index, out_base: str,
                              k: int = 50):
    out_dir = os.path.join(out_base, f"{sample_id}_sentences_hvg")
    os.makedirs(out_dir, exist_ok=True)

    ad = adata_sample[:, adata_sample.var_names.isin(hvgs)].copy()
    if ad.n_vars == 0:
        print(f"⚠️ {sample_id}: 无 HVG 交集, 跳过")
        return

    obs = ad.obs.copy()
    genes = ad.var_names.to_numpy()

    # 坐标鲁棒获取
    if 'pxl_col_in_fullres' not in obs.columns or 'pxl_row_in_fullres' not in obs.columns:
        if 'spatial' in ad.obsm:
            obs['pxl_col_in_fullres'] = ad.obsm['spatial'][:, 0]
            obs['pxl_row_in_fullres'] = ad.obsm['spatial'][:, 1]
        else:
            obs['pxl_col_in_fullres'] = np.arange(ad.n_obs)
            obs['pxl_row_in_fullres'] = 0

    for i in range(ad.n_obs):
        r, c = obs.iloc[i]['pxl_row_in_fullres'], obs.iloc[i]['pxl_col_in_fullres']
        suffix = f"{i}_0_no_coord" if (pd.isna(r) or pd.isna(c)) else f"{int(round(r))}_{int(round(c))}"
        path = os.path.join(out_dir, f"{sample_id}_{suffix}.txt")
        if os.path.exists(path):
            continue

        if issparse(ad.X):
            vec = ad.X.getrow(i).toarray().ravel()
        else:
            vec = np.asarray(ad.X[i]).ravel()

        idx = topk_indices(vec, k)
        toks = genes[idx]
        with open(path, "w") as f:
            f.write(" ".join(toks))

def validate_sentences_random(hvgs: set, out_base: str, sample_ids: list, n_checks: int = 20):
    """随机抽样句子文件，验证所有 token 均∈ HVG。"""
    import random, os
    sample_ids = sample_ids[:] if len(sample_ids) <= n_checks else random.sample(sample_ids, n_checks)
    ok = 0; bad = []
    for sid in sample_ids:
        d = os.path.join(out_base, f"{sid}_sentences_hvg")
        if not (os.path.exists(d) and os.listdir(d)):
            continue
        fname = random.choice(os.listdir(d))
        with open(os.path.join(d, fname), 'r') as f:
            toks = f.read().split()
        if all(t in hvgs for t in toks):
            ok += 1
        else:
            bad.append((sid, fname, [t for t in toks if t not in hvgs]))
    print(f"句子验证：通过 {ok}/{len(sample_ids)} 个样本。")
    if bad:
        print("非HVG token 示例（最多5例）：")
        for sid, fn, notin in bad[:5]:
            print(f"  - {sid}/{fn}: {notin[:10]} ...")


Seed=42; 输出目录=/cwStorage/nodecw_group/jijh/hest_sentences_hugo


In [9]:
# 2.3 NEW: 载入 HGNC 官方资源
CANONICAL_SET, SYN_MAP, SYMBOL2TYPE = load_hgnc_resources(
    HGNC_TSV, keep_status=KEEP_STATUS, keep_locus_types=KEEP_LOCUS_TYPES
)


HGNC 载入：官方符号 44459 个；同义/旧名映射 58030 条。


In [11]:
len(CANONICAL_SET)

44459

In [12]:
# 2.4 名称对齐 + 同义名映射 + 官方过滤 + 列聚合（核心修复）
adata = enforce_hgnc_and_collapse(
    adata, canonical_set=CANONICAL_SET, synonym_map=SYN_MAP, audit_dir=CACHE_DIR
)


/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


✅ 名称对齐完成：映射 4250；过滤 161962；最终基因 30716


In [13]:
# 2.5 QC 过滤 + 标准化
adata = qc_and_basic_filters(adata)
adata = normalize_log1p(adata)

✅ QC后: spots 1351225→1002113, genes 30716→20083 (min_cells=1002)


In [14]:
# 2.6 缓存
os.makedirs(os.path.dirname(CACHE_H5AD), exist_ok=True)
adata.write(CACHE_H5AD)
print(f"预处理后对象已缓存: {CACHE_H5AD}")

预处理后对象已缓存: /cwStorage/nodecw_group/jijh/hest_sentences_hugo/cache/adata_merged_preprocessed_canonical_collapsed.h5ad


In [ ]:
# 2.7 HVG（Seurat v3 paper, 批次感知）
print("计算 HVG ...")
hvgs = compute_hvgs(adata.copy(), n_top=N_TOP_HVGS, batch_key=BATCH_KEY, flavor='seurat_v3_paper')




计算 HVG ...


/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3_paper'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


AssertionError: HVG 中仍含非标准名（如 -1 后缀）: ['ERV3-1', 'H1-2', 'H1-3', 'H1-4', 'H1-5', 'NKX2-1', 'NKX3-1']

In [19]:
def validate_hvgs(hvgs: pd.Index,
                  canonical_set: set[str] | None,
                  symbol2type: dict[str, str] | None,
                  n_expected: int):
    # 1) 无重复
    assert not pd.Index(hvgs).duplicated().any(), "HVG 列表存在重复基因名"

    # 2) 与官方集合对齐（主判据）
    outside = []
    if canonical_set:
        # 注意：我们的管线已统一大写；为稳妥仍使用 upper() 对齐
        outside = [g for g in hvgs if g.upper() not in canonical_set]
        if outside:
            print(f"⚠️ 有 {len(outside)} 个 HVG 不在 HGNC 官方集合（示例）：{outside[:10]}")

    # 3) 仅对“不在官方集合”的基因再做后缀可疑性检测
    suspicious_suffix = [g for g in outside if re.search(r"-\d+$", g)]
    # —— 如果你希望对这类“强疑似 make_unique 伪后缀”严苛处理，用 assert；否则改成 warning：
    assert len(suspicious_suffix) == 0, (
        "HVG 中存在疑似唯一化伪后缀（不在HGNC且形如 `-数字`）的基因："
        f"{suspicious_suffix[:10]}"
    )

    # 4) 可选：locus_type 分布，便于审计（信息性输出，不做强校验）
    if symbol2type:
        types = pd.Series([symbol2type.get(g.upper(), "UNK") for g in hvgs]).value_counts()
        types.to_csv(os.path.join(CACHE_DIR, "hvg_locus_type_distribution.tsv"),
                     sep='\t', header=['count'])
        print("HVG locus_type 分布已写入 cache/hvg_locus_type_distribution.tsv")

    # 5) 数量提醒（不强制）
    if len(hvgs) != n_expected:
        print(f"⚠️ HVG 数量 {len(hvgs)} != 期望 {n_expected}。若前面做了严格过滤，这可能是正常的。")


In [20]:
# 2.8 HVG 验证 + 保存（对照官方集合与类型概览）
validate_hvgs(hvgs, canonical_set=CANONICAL_SET, symbol2type=SYMBOL2TYPE, n_expected=N_TOP_HVGS)
save_hvgs(hvgs, HVG_TXT)

print(f"HVG 完成: {len(hvgs)} genes; 用时 {format_time(time.time()-t0)}")

HVG locus_type 分布已写入 cache/hvg_locus_type_distribution.tsv
HVG 列表已保存: /cwStorage/nodecw_group/jijh/hest_sentences_hugo/global_hvgs.txt
HVG 完成: 5000 genes; 用时 55分 31.36秒


In [21]:
# =========================================================
# 3) 基于 HVG 的“基因句子”生成（NLP 化）
# =========================================================
print("\n开始生成基因句子（基于 HVG） ...")
from concurrent.futures import ThreadPoolExecutor

sid_list = adata.obs[BATCH_KEY].unique().tolist()

def _process_sid(sid: str):
    ad_s = adata[adata.obs[BATCH_KEY]==sid].copy()
    make_sentences_for_sample(sid, ad_s, hvgs, OUT_BASE, k=N_TOP_GENES_IN_SENT)

with ThreadPoolExecutor(max_workers=12) as ex:
    list(tqdm(ex.map(_process_sid, sid_list), total=len(sid_list)))

print("句子生成完毕。")

# 3.1 句子验证（抽样）
validate_sentences_random(set(hvgs), OUT_BASE, sid_list, n_checks=min(20, len(sid_list)))

print("全流程完成。")



开始生成基因句子（基于 HVG） ...


/public/home/jijh/micromamba/envs/gigapath/lib/python3.13/site-packages/anndata/_core/anndata.py:1022: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub[k] = df_sub[k].cat.remove_unused_categories()


  0%|          | 0/505 [00:00<?, ?it/s]

句子生成完毕。
句子验证：通过 20/20 个样本。
全流程完成。
